In [128]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import tensorflow as tf

In [129]:
models = tf.keras.models  # like 'from tensorflow.keras import models' (PyCharm import issue workaround)
layers = tf.keras.layers  # like 'from tensorflow.keras import layers' (PyCharm import issue workaround)
optimizers = tf.keras.optimizers  # like 'from tensorflow.keras import optimizers' (PyCharm import issue workaround)

In [130]:
def load_mnist_data():
    """
    Loads the MNIST Data Set and reshapes it for further model training

    :return:
        train_images        numpy array of shape (60000, 28, 28, 1)
        train_labels        numpy array of shape (60000, )
        test_images         numpy array of shape (10000, 28, 28, 1)
        test_labels         numpy array of shape (10000, )
    """

    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

    train_images = train_images.reshape((60000, 28, 28, 1))
    test_images = test_images.reshape((10000, 28, 28, 1))

    # Normalize pixel values to be between 0 and 1
    train_images, test_images = train_images / 255.0, test_images / 255.0

    return train_images, train_labels, test_images, test_labels

In [131]:
def build_cnn(input_shape):
    """
    Compile and return a simple CNN model for image recognition.

    Configuration:
    Layer 1: Convolution Layer | Filters: 32 | Kernel Size: 3x3 | Activation: Relu
    Layer 2: Max Pooling Layer | Filter: 2x2
    Layer 3: Dense Layer       | Neurons: 32 | Activation: Relu
    Layer 4: Dense Layer       | Neurons: 10 | Activation: Softmax

    Optimizer:      Adam
    Loss function:  Sparse Categorical Cross Entropy
    Loss metric:    Accuracy


    :param input_shape:     image input shape (tuple), e.g. (28, 28, 1)

    :return:
        model               compiled tensorflow model
    """

    # Set up model type
    model = models.Sequential()

    # Add layers, inspired by https://www.tensorflow.org/beta/tutorials/images/intro_to_cnns
    model.add(layers.Conv2D(32, (5, 5), input_shape=input_shape, kernel_initializer='random_normal'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (5, 5), input_shape=input_shape, kernel_initializer='random_normal'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu', kernel_initializer='random_normal'))
    model.add(layers.Dense(10, activation='softmax', kernel_initializer='random_normal'))

    return model

In [132]:
def Vname_to_FeedPname(var):
    return var.name[:var.name.find(':')] + '_placeholder:0'

In [133]:
class WeightsAccountant:
    def __init__(self, weights, clients, model):
        self.Global_Weights = weights
        self.Local_Weights = []
        self.Local_Updates = []
        self.Local_Norms = []
        self.Clipped_Updates = None
        self.Global_Updates = None
        self.median = None
        self.num_clients = clients
        
    def append_local_weights(self, weights):
        self.Local_Weights.append(weights)

    def average_local_weights(self):
        self.Local_Weights = np.array(self.Local_Weights).T
        self.Global_Weights = np.mean(self.Local_Weights, axis=1)
        del self.Local_Weights
        self.Local_Weights = []
        return self.Global_Weights

    def get_global_weights(self):
        return self.Global_Weights

    def compute_local_updates(self):
        for local_model in self.Local_Weights:
            updates = [glob - loc for glob, loc in zip(self.Global_Weights, local_model)]
            self.Local_Updates.append(updates)
        
    def compute_update_norm(self):
        self.Local_Norms = []
        for update in self.Local_Updates:
            l2_norm = [np.sqrt(np.sum(np.square(layer))) for layer in update]
            self.Local_Norms.append(l2_norm)
        self.Local_Norms = np.array(self.Local_Norms)
        
    def clip_updates(self):
        self.median = np.median(self.Local_Norms, axis=0)
        scaling_factor = np.divide(self.Local_Norms, self.median, where=self.median > 0)
        scaling_factor = np.maximum(scaling_factor, 1)
        self.Clipped_Updates = np.divide(self.Local_Updates, scaling_factor)

    def reset_local_parameters(self):
        del self.Local_Weights
        del self.Local_Updates
        del self.Local_Norms
        del self.Clipped_Updates
        self.Local_Weights = []
        self.Local_Updates = []
        self.Local_Norms = []
        self.Clipped_Updates = []

    def average_local_clipped_updates(self):
        # self.Local_Updates = np.array(self.Clipped_Updates).T
        self.Global_Updates = np.mean(np.array(self.Local_Updates).T, axis=1)
        self.reset_local_parameters()

    def compute_noisy_global_weights(self, sigma):
        self.Global_Weights = []
        for layer, median, weights in zip(self.Global_Updates, self.median, self.Global_Weights):
            noise = np.random.normal(loc=0, scale=sigma * median, size=layer.shape) / self.num_clients
            noise = 0
            noisy_update = layer + noise
            self.Global_Weights.append(noisy_update)

In [134]:
class WeightsAccountant_2:
    def __init__(self,sess,model,Sigma, real_round):

        self.Weights = [np.expand_dims(sess.run(v), -1) for v in tf.trainable_variables()]
        self.keys = [Vname_to_FeedPname(v) for v in tf.trainable_variables()]

        # The trainable parameters are [q x p] matrices, we expand them to [q x p x 1] in order to later stack them
        # along the third dimension.

        # Create a list out of the model dictionary in the order in which the graph holds them:

        self.global_model = [model[k] for k in self.keys]
        self.Sigma = Sigma
        self.Updates = []
        self.median = []
        self.Norms = []
        self.ClippedUpdates = []
        self.m = 0.0
        self.num_weights = len(self.Weights)
        self.round = real_round

    def save_params(self,save_dir):
        filehandler = open(save_dir + '/Wweights_accountant_round_'+self.round + '.pkl', "wb")
        pickle.dump(self, filehandler)
        filehandler.close()

    def allocate(self, sess):

        self.Weights = [np.concatenate((self.Weights[i], np.expand_dims(sess.run(tf.trainable_variables()[i]), -1)), -1)
                        for i in range(self.num_weights)]

        # The trainable parameters are [q x p] matrices, we expand them to [q x p x 1] in order to stack them
        # along the third dimension to the already allocated older variables. We therefore have a list of 6 numpy arrays
        # , each numpy array having three dimensions. The last dimension is the one, the individual weight
        # matrices are stacked along.

    def compute_updates(self):

        # To compute the updates, we subtract the global model from each individual weight matrix. Note:
        # self.Weights[i] is of size [q x p x m], where m is the number of clients whose matrices are stored.
        # global_model['i'] is of size [q x p], in order to broadcast correctly, we have to add a dim.

        self.Updates = [self.Weights[i]-np.expand_dims(self.global_model[i], -1) for i in range(self.num_weights)]
        self.Weights = None

    def compute_norms(self):

        # The norms List shall have 6 entries, each of size [1x1xm], we keep the first two dimensions because
        # we will later broadcast the Norms onto the Updates of size [q x p x m]

        self.Norms = [np.sqrt(np.sum(
            np.square(self.Updates[i]), axis=tuple(range(self.Updates[i].ndim)[:-1]),keepdims=True)) for i in range(self.num_weights)]

    def clip_updates(self):
        self.compute_updates()
        self.compute_norms()

        # The median is a list of 6 entries, each of size [1x1x1],

        self.median = [np.median(self.Norms[i], axis=-1, keepdims=True) for i in range(self.num_weights)]

        # The factor is a list of 6 entries, each of size [1x1xm]

        factor = [self.Norms[i]/self.median[i] for i in range(self.num_weights)]
        for i in range(self.num_weights):
            factor[i][factor[i] > 1.0] = 1.0

        self.ClippedUpdates = [self.Updates[i]/factor[i] for i in range(self.num_weights)]

    def Update_via_GaussianMechanism(self, sess, Acc, FLAGS, Computed_deltas):
        self.clip_updates()
        self.m = float(self.ClippedUpdates[0].shape[-1])
        MeanClippedUpdates = [np.mean(self.ClippedUpdates[i], -1) for i in range(self.num_weights)]

        GaussianNoise = [(1.0/self.m * np.random.normal(loc=0.0, scale=float(self.Sigma * self.median[i]), size=MeanClippedUpdates[i].shape)) for i in range(self.num_weights)]
        for update, noise, median in zip(MeanClippedUpdates, GaussianNoise, self.median):
            print("Median", median)

        print("m:", self.m)
        Sanitized_Updates = [MeanClippedUpdates[i]+GaussianNoise[i] for i in range(self.num_weights)]

        New_weights = [self.global_model[i]+Sanitized_Updates[i] for i in range(self.num_weights)]

        New_model = dict(zip(self.keys, New_weights))

        t = Acc.accumulate_privacy_spending(0, self.Sigma, self.m)
        delta = 1
        if FLAGS.record_privacy == True:
            if FLAGS.relearn == False:
                # I.e. we never learned a complete model before and have therefore never computed all deltas.
                for j in range(len(self.keys)):
                    sess.run(t)
                r = Acc.get_privacy_spent(sess, [FLAGS.eps])
                delta = r[0][1]
            else:
                # I.e. we have computed a complete model before and can reuse the deltas from that time.
                delta = Computed_deltas[self.round]
        return New_model, delta


In [141]:
model_placeholder = dict(zip([Vname_to_FeedPname(var) for var in tf.compat.v1.trainable_variables()],
                                 [tf.placeholder(name=Vname_to_Pname(var),
                                                 shape=var.shape,
                                                 dtype=tf.float32)
                                  for var in tf.compat.v1.trainable_variables()]))

In [144]:
assignments = [tf.assign(var, model_placeholder[Vname_to_FeedPname(var)]) for var in
                   tf.compat.v1.trainable_variables()]

In [146]:
model = []

In [147]:
init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)
weights_accountant_2 = WeightsAccountant(sess, model, sigma, real_round)

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().

In [114]:
# Build model
model_1 = build_cnn(input_shape=(28, 28, 1))
model_2 = build_cnn(input_shape=(28, 28, 1))
model_3 = build_cnn(input_shape=(28, 28, 1))
model_4 = build_cnn(input_shape=(28, 28, 1))
# Compile the model
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [115]:
weights_1 = model_1.get_weights()
weights_2 = model_2.get_weights()
weights_3 = model_3.get_weights()
weights_4 = model_4.get_weights()
acc = WeightsAccountant(weights_1, 100, model_1)
acc.append_local_weights(weights_2)
acc.append_local_weights(weights_3)
acc.append_local_weights(weights_4)

In [116]:
acc.compute_local_updates()
acc.compute_update_norm()
acc.clip_updates()
acc.average_local_clipped_updates()
acc.compute_noisy_global_weights(1)

AttributeError: 'numpy.ndarray' object has no attribute 'sqrt'